In [1]:
import pandas as pd
from time import sleep
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as BS

In [2]:
## Setup chrome options
chrome_options = Options()
chrome_options.add_argument("--headless") # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")

# webdriver_service = Service(ChromeDriverManager().install())
webdriver_service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service = webdriver_service, options = chrome_options)

In [3]:
url = "https://subway.com.my/find-a-subway"
driver.get(url)

# find the search box
search_box = driver.find_element(By.ID, "fp_searchAddress")
search_box.clear()
search_box.send_keys("kuala lumpur")
keys = Keys.RETURN
search_box.send_keys(keys)

loc_detail = driver.find_elements(by=By.CLASS_NAME, value='fp_listitem')

In [4]:
outlet_location = []
for loc in loc_detail:
    loc_lat = loc.get_attribute('data-latitude')
    loc_long = loc.get_attribute('data-longitude')
    loc_name = loc.find_element(By.TAG_NAME, 'h4').text
    loc_address = loc.find_element(By.CSS_SELECTOR, '.infoboxcontent p:nth-child(1)').text  # Selects first paragraph
    # loc_operating = loc.find_element(By.CSS_SELECTOR, '.infoboxcontent p:nth-child(3)').text  # Selects third paragraph
    loc_waze = loc.find_element(By.CSS_SELECTOR, '.fa-brands.fa-waze').find_element(By.XPATH, '..').get_attribute('href')
    try:
        loc_operating = loc.find_element(By.CSS_SELECTOR, '.infoboxcontent p:nth-child(3)').text
    except Exception as e:
        loc_operating = "Operating hours not available"
        
    outlet_location.append({'Latitude': loc_lat,'Longitude':loc_long,'Name':loc_name,'Address':loc_address,'Operating_Hours':loc_operating, 'Waze': loc_waze})

In [5]:
import numpy as np
df=pd.DataFrame(outlet_location)

df = df.replace('', np.nan).dropna()
df.to_csv('subway_location.csv', index=False)
driver.quit()
df

,Latitude,Longitude,Name,Address,Operating_Hours,Waze
0,3.128099,101.678678,Subway Menara UOA Bangsar,"Jalan Bangsar Utama 1, Unit 1-2-G, Menara UOA ...","Monday - Sunday, 8:00 AM - 8:00 PM",https://www.waze.com/en/live-map/directions/my...
1,3.151251,101.615116,Subway One Utama,"318A One Utama, Lower Ground Floor, New Wing, ...",0800 - 2200 (Sun - Thur),https://www.waze.com/en/live-map/directions/my...
2,3.152705,101.712034,Subway Jln Pinang,"G9, Wisma UOA II, 19, Jalan Pinang, Kuala Lump...","Monday - Saturday, 8:00 AM – 9:00PM",https://www.waze.com/en/live-map/directions/my...
3,3.151729,101.666006,Subway UOA Damansara,"Unit 50-G-5, Ground Floor, Wisma UOA Damansara...","Monday - Saturday, 8:00 AM – 8:30PM",https://www.waze.com/en/live-map/directions/my...
4,3.027692,101.718905,Subway The Mines,"Lot L1-93, Level 1, Mines Shopping Fair,Jln Du...","Monday - Sunday, 10:00 AM - 10:00 PM",https://www.waze.com/en/live-map/directions/su...
...,...,...,...,...,...,...
346,3.143814,101.741630,Subway Lotus Ampang,"LOTUS’S AMPANG - OL1, Jalan Pandan Prima Datar...",Monday - Sunday (10:00AM - 10:00PM),"https://waze.com/ul?ll=3.143814,101.741630"
349,3.108465,101.706308,Subway Razak City,"Block B-G-19 Residensi Bandar Razak, No 01 Jln...",Monday - Sunday (08:00AM to 10:00PM),"https://waze.com/ul?ll=3.108465,101.706308"
351,3.033015,101.765206,Subway Aeon Cheras Selatan,"F60, First Floor, ÆON Cheras Selatan, Lebuh Ut...",Monday to Sunday (10:00AM - 10:00PM),"https://waze.com/ul?ll=3.033015,101.765206"
359,3.226999,101.650259,Subway Kiara Bay,"""Lot G-22, The Beat Kiara Bay Kepong Metropoli...",Monday - Sunday (10:00AM - 10:00PM),"https://waze.com/ul?ll=3.226999,101.650259"
